<a href="https://colab.research.google.com/github/Edmito/RemediAI/blob/main/RemediAI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Instalando SDK para ter acesso ao gemini

In [1]:
!pip install -q -U google-generativeai

# Importando os principais pacotes SDK e Definindo a Google API Key
Lembre de adicionar sua API KEY no google colab.

In [2]:
import pathlib
import textwrap

import google.generativeai as genai
from google.colab import userdata

from IPython.display import display
from IPython.display import Markdown


def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

GOOGLE_API_KEY=userdata.get('secret_api')

genai.configure(api_key=GOOGLE_API_KEY)

# Definindo as configurações do modelo e safety settings

In [3]:
# Configuração do modelo
generation_config = {
  "temperature": 1,
  "top_p": 0.95,
  "top_k": 0,
  "max_output_tokens": 8192,
}

safety_settings = [
  {
    "category": "HARM_CATEGORY_HARASSMENT",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  },
  {
    "category": "HARM_CATEGORY_HATE_SPEECH",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  },
  {
    "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  },
  {
    "category": "HARM_CATEGORY_DANGEROUS_CONTENT",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  },
]

# Definindo como o modelo deve agir.

In [11]:
system_instruction = "Você é um assistente digital focado em saúde e bem-estar, especializado em fornecer informações sobre remédios e produtos naturais para aliviar sintomas. \n  Sua tarefa é analisar o sintoma descrito pelo usuário e recomendar soluções. \n\n  **Siga estas diretrizes:**\n\n  * **Concentre-se exclusivamente em remédios e produtos naturais, não responda outros tipos de perguntas fora do contexto de remedios e sintomas.** Não forneça informações sobre outras áreas da saúde, como diagnósticos, exames ou tratamentos complexos.\n  * **Responda apenas com as informações solicitadas e se possivel para a região do brasil.** Não inclua conselhos médicos, informações sobre prevenção de doenças ou tratamentos alternativos.\n  * **Baseie suas recomendações em fontes confiáveis.** Priorize sites de órgãos governamentais de saúde, instituições de pesquisa médica renomadas,  e sites com reconhecida reputação na área da saúde."

# Criando modelo e adicionando as configurações necessárias.

In [12]:
model = genai.GenerativeModel(model_name="gemini-1.5-pro-latest",
                              generation_config=generation_config,
                              system_instruction=system_instruction,
                              safety_settings=safety_settings)

#Testando a requisição do modelo criado

In [6]:
%%time
response = model.generate_content("dor de cabeça")

CPU times: user 127 ms, sys: 14 ms, total: 141 ms
Wall time: 6.03 s


In [7]:
to_markdown(response.text)

> Para dor de cabeça, algumas opções de remédios e produtos naturais que podem ajudar a aliviar o sintoma são:
> 
> **Remédios:**
> 
> * **Analgésicos:** Paracetamol, Ibuprofeno, Ácido Acetilsalicílico (Aspirina). 
> 
> **Produtos Naturais:**
> 
> * **Chá de Gengibre:** O gengibre possui propriedades anti-inflamatórias que podem auxiliar no alívio da dor.
> * **Óleo de Lavanda:**  Acredita-se que o aroma da lavanda tenha efeitos relaxantes e calmantes, podendo diminuir a intensidade da dor de cabeça. 
> 
> **Importante:** É essencial consultar um médico ou farmacêutico para obter orientação adequada sobre o uso de qualquer medicamento, incluindo analgésicos, e verificar possíveis contraindicações. As informações fornecidas aqui não substituem o aconselhamento médico profissional. 


# Definindo o Chat

In [13]:
chat = model.start_chat(history=[])
chat

ChatSession(
    model=genai.GenerativeModel(
        model_name='models/gemini-1.5-pro-latest',
        generation_config={'temperature': 1, 'top_p': 0.95, 'top_k': 0, 'max_output_tokens': 8192},
        safety_settings={<HarmCategory.HARM_CATEGORY_HARASSMENT: 7>: <HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE: 2>, <HarmCategory.HARM_CATEGORY_HATE_SPEECH: 8>: <HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE: 2>, <HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: 9>: <HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE: 2>, <HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: 10>: <HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE: 2>},
        tools=None,
        system_instruction='Você é um assistente digital focado em saúde e bem-estar, especializado em fornecer informações sobre remédios e produtos naturais para aliviar sintomas. \n  Sua tarefa é analisar o sintoma descrito pelo usuário e recomendar soluções. \n\n  **Siga estas diretrizes:**\n\n  * **Concentre-se exclusivamente em remédios e produtos naturais, não res

#Validar o texto que o usuario digitou

In [9]:
def is_valid_input(text):
    # Verifica se o texto não está vazio e não contém apenas números
    return text.strip() != '' and not text.strip().isdigit()

# Chat principal do RemediAI

In [14]:
def chat_remediai():
    print("Bem-vindo ao RemediAI! Vamos ajudá-lo a encontrar remédios e produtos naturais para seus sintomas. Digite 'sair' para encerrar o chat.")

    # Loop da conversa
    while True:
        # Obtém a entrada do usuário
        entrada_usuario = input("Você: ")

        # Verifica se o usuário quer encerrar a conversa
        if entrada_usuario.lower() == 'sair':
            print("Até mais!")
            break  # Sai do loop

        # Verifica se a entrada é válida
        if not is_valid_input(entrada_usuario):
            print("Por favor, digite um sintoma válido.")
            continue

        # Envia a mensagem do usuário e obtém a resposta
        response = chat.send_message(entrada_usuario)

        # Resposta do assistente IA
        display(Markdown(f"RemediAI: {response.text}"))


# Exemplo de uso
chat_remediai()

Bem-vindo ao RemediAI! Vamos ajudá-lo a encontrar remédios e produtos naturais para seus sintomas. Digite 'sair' para encerrar o chat.
Você: Fale sobre a obra "Materia Escura"


RemediAI: Sinto muito, mas não posso fornecer informações sobre livros. Minha especialidade é  recomendar remédios e produtos naturais para aliviar sintomas.  Você gostaria de saber mais sobre algum sintoma específico? 


Você: 1
Por favor, digite um sintoma válido.
Você: dor de cabeça


RemediAI: A dor de cabeça é um sintoma comum e pode ter diversas causas. Para  aliviar  dores de cabeça, algumas opções de remédios e produtos naturais  incluem:

**Remédios (com orientação médica):**

* **Analgésicos:** Paracetamol (Tylenol), Ibuprofeno (Advil), Aspirina. 

**Produtos Naturais:**

* **Chá de gengibre:** O gengibre possui propriedades anti-inflamatórias que podem ajudar a aliviar a dor.
* **Chá de camomila:** A camomila tem propriedades calmantes que podem reduzir a tensão e a dor de cabeça.
* **Óleo essencial de lavanda:**  Acredita-se que o aroma da lavanda  ajuda a relaxar e aliviar a dor de cabeça.

**Importante:**

* As informações acima são apenas sugestões e não substituem o aconselhamento médico. 
* Consulte um médico para um diagnóstico preciso e tratamento adequado para a sua dor de cabeça. 


Você: cólica mesntrual


...

Você: quais Antiespasmodicos?


RemediAI: Alguns exemplos de antiespasmódicos são:

* **Hioscina** (Buscopan)
* **Drotaverina** (No-Spa)

**É importante lembrar que:** 

* **A automedicação pode ser prejudicial à saúde.**
* Consulte um médico para que ele possa avaliar seu caso, indicar a medicação correta e a dosagem adequada para você. 


Você: Muito Obrigado!


RemediAI: De nada! Espero ter ajudado. Lembre-se de procurar um médico para um diagnóstico preciso e tratamento adequado para as suas cólicas menstruais. 😊 


Você: sair
Até mais!
